### Target Variable : Tip with mutate variable -> tip_rate

In [ ]:
!cd

In [ ]:
!dir

In [ ]:
#!pip install PyMySQL==1.0.0

In [ ]:
#import pymysql.cursors
import pandas as pd
# Connect to the database

In [ ]:
# localhost or 127.0.0.1

In [ ]:
'''
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='8475', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

    with conn.cursor() as curs:
      sql = "SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)
      #df.to_csv('query.csv')

finally:

   conn.close()
'''

In [ ]:
df = pd.read_csv('tips.csv')

In [ ]:
df.info()

In [ ]:
#df.isnull().sum()

### Encoding

In [ ]:
df['sex'].replace({'Female':0, 'Male':1}, inplace=True)
df["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)
df["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
import numpy as np

In [ ]:
df.replace('',np.nan,inplace=True) # '' -> np.nan

In [ ]:
df.describe()

In [ ]:
type(df)

In [ ]:
df['tip_rate'] = ((df['tip']/df['total_bill'])*100).round(2)

In [ ]:
df['tip_rate'].head(3)

In [ ]:
df.describe()

In [ ]:
#import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
data = df.values

In [ ]:
y=df['tip'] # Target variable -> Tip

In [ ]:
X=df.drop('tip',axis=1)

In [ ]:
X.columns

In [ ]:
y.describe()

In [ ]:
data=X.values # Train data

In [ ]:
y=y.values

In [ ]:
import numpy as np

In [ ]:
sum(np.isnan(data).flatten())

In [ ]:
sum(np.isnan(y).flatten())

In [ ]:
imputer = SimpleImputer(strategy='median')

In [ ]:
imputer.fit(data)

In [ ]:
# transform the dataset
data_trans = imputer.transform(data) # Imputed data set

### 시작 

### RFE

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
#from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.svm import SVR

In [ ]:
estimator = SVR(kernel="linear")

In [ ]:
# define RFE
rfe = RFE(estimator, n_features_to_select=4)

In [ ]:
# fit RFE
selector=rfe.fit(data_trans, y)

In [ ]:
!cd

In [ ]:
selector.support_

In [ ]:
# summarize all features
for i in range(data_trans.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

In [ ]:
df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size'])

In [ ]:
df1.columns

In [ ]:
df1.drop(['smoker', 'day'], axis=1)

### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
data_trans.shape

In [ ]:
# define the transform
trans = PCA(n_components=4)

In [ ]:
# https://stackoverflow.com/questions/55582413/put-results-of-pca-into-dataframe

In [ ]:
# transform the data
X_dim = trans.fit_transform(data_trans)

In [ ]:
# summarize data after the transform
print(X_dim[:3, :])

In [ ]:
trans.components_

In [ ]:
trans.explained_variance_ratio_

In [ ]:
df_pca = pd.DataFrame(X_dim)

In [ ]:
df_pca.describe().round(2)

In [ ]:
df.describe()

### Regression Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
fs = SelectKBest(score_func=f_regression, k=4)

In [ ]:
data_trans.shape

In [ ]:
y.shape

In [ ]:
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

In [ ]:
parameters = fs.get_params(deep=True)

In [ ]:
parameters

In [ ]:
parameters['score_func']

In [ ]:
get_features = fs.fit(data_trans, y)

In [ ]:
get_features

In [ ]:
fs.get_support(indices=True)

In [ ]:
print(X_selected.shape)

In [ ]:
X_selected.shape[1]

In [ ]:
df_rfe_selected = pd.DataFrame(X_selected, columns=['total_bill', 'day', 'time', 'size'])

In [ ]:
df_rfe_selected.describe()